In [1]:
import pandas as pd
import importlib.util
import sklearn
import numpy as np
from pathlib import Path
import joblib
import os   

In [2]:
output_dir = "classification_results_2023/"
all_patient_data = pd.read_csv("Combined_AM_Data.csv")

In [3]:
all_patient_data.columns

Index(['id', 'date', 'age', 'sex', 'BMI', 'income', 'road_dist', 'cooking',
       'am_pef_org', 'y_am_pef', 'tempin', 'humidin', 'pm25in', 'co2in',
       'tempdiffin', 'humidiffin', 'pm25diffin', 'pm10', 'pm25', 'o3', 'no2',
       'co', 'so2', 'temp', 'windsd', 'humid', 'varp', 'dewpt', 'airp', 'seap',
       'solrhr', 'solramnt', 'grdt', 'class'],
      dtype='object')

In [4]:
# Loading classification.py
spec = importlib.util.spec_from_file_location('classification', r"Classificationv3.py")
module = importlib.util.module_from_spec(spec)
spec.loader.exec_module(module)

In [5]:
predictColumn = "class"
minorityVariable = 0 
classes = all_patient_data[predictColumn].unique().tolist()

n_classes = len(classes)

if n_classes > 2 :
    logit_reg_model = sklearn.linear_model.LogisticRegression(solver='saga', multi_class='multinomial')
else :
    logit_reg_model = sklearn.linear_model.LogisticRegression(solver='liblinear', multi_class='auto')

cls = [ "DecisionTree" , sklearn.tree.DecisionTreeClassifier(max_depth=4),
        "K-NN"              , sklearn.neighbors.KNeighborsClassifier(),
        "Logit Regression"  , logit_reg_model,
        "Naive Bayes"       , sklearn.gaussian_process.GaussianProcessClassifier()
];

In [6]:
EVALUATION_METRICS = ['Weighted Accuracy', 'Sensitivity/Recall', 'Precision_avg', 'F1_avg', 'Specificity', 'Precision', 'Precision_class1', 'F1', 'F1_class1']
REQUIRED_COLUMNS = ['income','road_dist','cooking','y_am_pef','tempin','humidin','pm25in','co2in','tempdiffin','humidiffin','pm25diffin','pm10','pm25','o3','no2','co','so2','temp','windsd','humid','varp','dewpt','airp','seap','solrhr','solramnt','grdt','class']

In [7]:
%run "Classificationv3.py"
%run "Balance.py"

In [8]:
def classification_func(file_name, predictCol, minorVar, category_group, cls, output_dir, df_evaluation_results, patient_id, patient_id_df):

    print('~~~~~~~~~~~~~~~~~', file_name,'~~~~~~~~~~~~~~~~~~~~~~')

    dataset_dir = Path(output_dir + patient_id+category_group +  "/diagrams/" + file_name + '/') 

    # Create Directory for this patient
    if not (dataset_dir.exists() and dataset_dir.is_dir()):
        os.makedirs(dataset_dir)

    save_path = output_dir+ patient_id+category_group +'/diagrams/' + file_name + '/sklearn_models_balanced_for_train_'

    CLASS_VAR = predictCol
    MINORITY_VAR = minorVar
    bestDT = module.Classify(patient_id_df, True, class_var=CLASS_VAR, minority_var=MINORITY_VAR, drawConfusionMatrix=True, classifiers=cls, scale=True, save_path=save_path, after_split=True)

    confusion_matrixes_file = save_path + "confusion_mat_" + category_group + "_"+ file_name
    module.save_confusion_matrix(bestDT[1], confusion_matrixes_file)
    roc_curve_file = save_path + "roc_curve_" + category_group + "_"+ file_name
    module.save_Draw_Roc(bestDT[2], roc_curve_file)
    if len(bestDT[3])!=0:
      featureImportance_path = save_path + "important_mat_" + category_group + "_"+ file_name
      module.save_Draw_FeatureImportanceMatrix(patient_id_df,bestDT[3],cls[0], featureImportance_path, True)

    model_path = save_path + "bestModel_" + category_group + "_"+ file_name + ".joblib"
    joblib.dump(bestDT[4], model_path)

    df_evaluation_results[file_name] = bestDT[0]
    df_evaluation_results.to_csv(output_dir+patient_id+category_group+"/sklearn_models_balanced_for_train_metrics_"+category_group + ".csv")


In [9]:
def runner_decision_tree():
    ids = all_patient_data['id'].unique().tolist()
    for id in ids:
        patient_id = id + "/"
        file_name = id
        patient_id_df = all_patient_data[all_patient_data['id'] == id]
        filtered_df = patient_id_df.filter(REQUIRED_COLUMNS)
        patient_id_df = filtered_df.copy()
        
        # Decision Tree
        df_evaluation_results = pd.DataFrame(index=EVALUATION_METRICS)
        category_group = '1DecisionTree'
        cls = ["DecisionTree", sklearn.tree.DecisionTreeClassifier(max_depth=4)]
        classification_func(file_name, predictColumn, minorityVariable, category_group, cls, output_dir, df_evaluation_results, patient_id, patient_id_df)

def runner_knn():
    ids = all_patient_data['id'].unique().tolist()
    for id in ids:
        patient_id = id + "/"
        file_name = id
        patient_id_df = all_patient_data[all_patient_data['id'] == id]
        filtered_df = patient_id_df.filter(REQUIRED_COLUMNS)
        patient_id_df = filtered_df.copy()
        
        # KNN
        df_evaluation_results = pd.DataFrame(index=EVALUATION_METRICS)
        category_group = '2K-NN'
        cls = ["K-NN", sklearn.neighbors.KNeighborsClassifier()]
        classification_func(file_name, predictColumn, minorityVariable, category_group, cls, output_dir, df_evaluation_results, patient_id, patient_id_df)

def runner_lr():
    ids = all_patient_data['id'].unique().tolist()
    for id in ids:
        patient_id = id + "/"
        file_name = id
        patient_id_df = all_patient_data[all_patient_data['id'] == id]
        filtered_df = patient_id_df.filter(REQUIRED_COLUMNS)
        patient_id_df = filtered_df.copy()
        
        # Logistic Regression
        df_evaluation_results = pd.DataFrame(index=EVALUATION_METRICS)
        category_group = 'LogisticRegression'
        print(logit_reg_model)
        cls = ["Logit Regression", logit_reg_model]
        classification_func(file_name, predictColumn, minorityVariable, category_group, cls, output_dir, df_evaluation_results, patient_id, patient_id_df)

def runner_svm():
    ids = all_patient_data['id'].unique().tolist()
    for id in ids:
        patient_id = id + "/"
        file_name = id
        patient_id_df = all_patient_data[all_patient_data['id'] == id]
        filtered_df = patient_id_df.filter(REQUIRED_COLUMNS)
        patient_id_df = filtered_df.copy()
        
        # SVM
        df_evaluation_results = pd.DataFrame(index=EVALUATION_METRICS)
        category_group = 'SVM'
        cls = ["SVM", sklearn.svm.SVC(kernel="linear")]
        classification_func(file_name, predictColumn, minorityVariable, category_group, cls, output_dir, df_evaluation_results, patient_id, patient_id_df)

def runner_nb():
    ids = all_patient_data['id'].unique().tolist()
    for id in ids:
        patient_id = id + "/"
        file_name = id
        patient_id_df = all_patient_data[all_patient_data['id'] == id]
        filtered_df = patient_id_df.filter(REQUIRED_COLUMNS)
        patient_id_df = filtered_df.copy()
        
        # Naive Bayes
        df_evaluation_results = pd.DataFrame(index=EVALUATION_METRICS)
        category_group = 'NaiveBayes'
        cls = ["Naive Bayes", sklearn.gaussian_process.GaussianProcessClassifier()]
        classification_func(file_name, predictColumn, minorityVariable, category_group, cls, output_dir, df_evaluation_results, patient_id, patient_id_df)


In [10]:
#runner_decision_tree()

In [11]:
runner_knn()

~~~~~~~~~~~~~~~~~ SB-001 ~~~~~~~~~~~~~~~~~~~~~~
~~~~~~~~~~ Number of instances before balancing:  125  ~~~~~~~~~~
~~~~~~~ Training Loss: 0.008071603253483772  ~~~~~~~
~~~~~~~ Validation Loss: 0.011220409534871578  ~~~~~~~
2/2 [==============================] - 0s 7ms/step
~~~~~~~ value_counts: ~~~~~~~
1    87
0    76
Name: class, dtype: int64
~~~~~~~~~~ Number of instances after balancing:  163  ~~~~~~~~~~
~~~~~~~~~~ Number of instances before balancing:  125  ~~~~~~~~~~
~~~~~~~ Training Loss: 0.011599536053836346  ~~~~~~~
~~~~~~~ Validation Loss: 0.009099342860281467  ~~~~~~~
2/2 [==============================] - 0s 4ms/step
~~~~~~~ value_counts: ~~~~~~~
0    84
1    83
Name: class, dtype: int64
~~~~~~~~~~ Number of instances after balancing:  167  ~~~~~~~~~~
~~~~~~~~~~ Number of instances before balancing:  126  ~~~~~~~~~~
~~~~~~~ Training Loss: 0.013307472690939903  ~~~~~~~
~~~~~~~ Validation Loss: 0.017320284619927406  ~~~~~~~
2/2 [==============================] - 0s 7ms/step
~~~

c:\Users\sbhuv\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\neighbors\_classification.py:215: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
c:\Users\sbhuv\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\neighbors\_classification.py:215: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


* 1-----------kfold--------------- 2 ------------
---K-NN--- 0.75 KNeighborsClassifier(metric='manhattan')
* 2*
{'algorithm': 'auto', 'leaf_size': 30, 'metric': 'manhattan', 'metric_params': None, 'n_jobs': None, 'n_neighbors': 5, 'p': 2, 'weights': 'uniform'}
------run_cv runs successfully------------
++++++++++++save_confusion_matrix++++++++++++


c:\Users\sbhuv\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\neighbors\_classification.py:215: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
Classificationv3.py:353: UserWarning: FixedFormatter should only be used together with FixedLocator
  axs[i].set_xticklabels([''] + ['0', '1'], fontsize='xx-large')
Classificationv3.py:354: UserWarning: FixedFormatter should only be used together with FixedLocator
  axs[i].set_yticklabels([''] + ['0', '1'], fontsize='xx-large')


++++++++++++save_confusion_matrix end++++++++++++
(((((((((((((((((((((( draw droc ))))))))))))))))))))))))))))))
(((((((( draw droc end ))))))))
~~~~~~~~~~~~~~~~~ SB-003 ~~~~~~~~~~~~~~~~~~~~~~
~~~~~~~~~~ Number of instances before balancing:  120  ~~~~~~~~~~
~~~~~~~ Training Loss: 0.013222628273069859  ~~~~~~~
~~~~~~~ Validation Loss: 0.023398708552122116  ~~~~~~~
1/1 [==============================] - 1s 572ms/step
~~~~~~~ value_counts: ~~~~~~~
1    89
0    62
Name: class, dtype: int64
~~~~~~~~~~ Number of instances after balancing:  151  ~~~~~~~~~~
~~~~~~~~~~ Number of instances before balancing:  120  ~~~~~~~~~~
~~~~~~~ Training Loss: 0.014282937161624432  ~~~~~~~
~~~~~~~ Validation Loss: 0.027502205222845078  ~~~~~~~
1/1 [==============================] - 0s 279ms/step
~~~~~~~ value_counts: ~~~~~~~
1    93
0    54
Name: class, dtype: int64
~~~~~~~~~~ Number of instances after balancing:  147  ~~~~~~~~~~
~~~~~~~~~~ Number of instances before balancing:  120  ~~~~~~~~~~
~~~~~~~ Trai

...

* 2*
{'algorithm': 'auto', 'leaf_size': 30, 'metric': 'euclidean', 'metric_params': None, 'n_jobs': None, 'n_neighbors': 3, 'p': 2, 'weights': 'uniform'}
------run_cv runs successfully------------
++++++++++++save_confusion_matrix++++++++++++
++++++++++++save_confusion_matrix end++++++++++++
(((((((((((((((((((((( draw droc ))))))))))))))))))))))))))))))
(((((((( draw droc end ))))))))
~~~~~~~~~~~~~~~~~ SB-112 ~~~~~~~~~~~~~~~~~~~~~~
~~~~~~~~~~ Number of instances before balancing:  120  ~~~~~~~~~~
~~~~~~~ Training Loss: 0.017545949667692184  ~~~~~~~
~~~~~~~ Validation Loss: 0.031054122373461723  ~~~~~~~
1/1 [==============================] - 0s 165ms/step
~~~~~~~ value_counts: ~~~~~~~
1    95
0    50
Name: class, dtype: int64
~~~~~~~~~~ Number of instances after balancing:  145  ~~~~~~~~~~
~~~~~~~~~~ Number of instances before balancing:  120  ~~~~~~~~~~
~~~~~~~ Training Loss: 0.010115372017025948  ~~~~~~~
~~~~~~~ Validation Loss: 0.01588032953441143  ~~~~~~~
1/1 [=====================

c:\Users\sbhuv\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\neighbors\_classification.py:215: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
c:\Users\sbhuv\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\neighbors\_classification.py:215: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


* 1-----------kfold--------------- 2 ------------
---K-NN--- 0.8206896551724139 KNeighborsClassifier(metric='manhattan', n_neighbors=3)
* 2*
{'algorithm': 'auto', 'leaf_size': 30, 'metric': 'manhattan', 'metric_params': None, 'n_jobs': None, 'n_neighbors': 3, 'p': 2, 'weights': 'uniform'}
------run_cv runs successfully------------
++++++++++++save_confusion_matrix++++++++++++


c:\Users\sbhuv\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\neighbors\_classification.py:215: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
Classificationv3.py:353: UserWarning: FixedFormatter should only be used together with FixedLocator
  axs[i].set_xticklabels([''] + ['0', '1'], fontsize='xx-large')
Classificationv3.py:354: UserWarning: FixedFormatter should only be used together with FixedLocator
  axs[i].set_yticklabels([''] + ['0', '1'], fontsize='xx-large')


++++++++++++save_confusion_matrix end++++++++++++
(((((((((((((((((((((( draw droc ))))))))))))))))))))))))))))))
(((((((( draw droc end ))))))))


In [12]:
# runner_lr()

In [13]:
# runner_svm()

In [14]:
# runner_nb()